<br>

In [ ]:
#| include: false
from fastai.vision.all import *
from fasterai.misc.all import *

import torch
import torch.nn as nn
import torch.nn.functional as F

## 1. Get the data

In [ ]:
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")

def label_func(f): return f[0].isupper()

dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(64))

## 2. Train the model

In [ ]:
learn = Learner(dls, vgg16_bn(num_classes=2), metrics=accuracy)
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,0.886644,0.652151,0.685386,00:22
1,0.692583,0.627857,0.685386,00:21
2,0.646516,0.622866,0.685386,00:22


3. Decompose !

In [ ]:
#| include: false
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [ ]:
fc = FC_Decomposer()
new_model = fc.decompose(learn.model)

The fc layers have been factorized and replace by smaller ones.

In [ ]:
new_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

We can see compare the amount of parameters before/after:

In [ ]:
count_parameters(learn.model)

134277186

In [ ]:
count_parameters(new_model)

91281476

This represents a decrease of ~40M parameters ! 

Now this is an approximation, so it isn't really lossless and we should expect to see a performance drop, which will be bigger as we keep fewer singular values. Here we have:

In [ ]:
new_learn = Learner(dls, new_model, metrics=accuracy)
new_learn.validate()

(#2) [0.6868855357170105,0.6853856444358826]